In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
import string
import random


def create_random_string(n):
    randstring = "".join(random.choice(string.printable) for _ in range(n))
    while (toklen := len(encoding.encode(randstring))) < n:
        randstring += "".join(random.choice(string.ascii_letters) for _ in range(n - toklen))
    return randstring

In [ ]:
a = create_random_string(20000)
len(encoding.encode(a))

In [ ]:
from openai import OpenAI

client = OpenAI(organization="org-dxkPAvuWroRmYw7z0VhqMC5S")

In [ ]:
resp = client.chat.completions.create(messages=[{"role": "user", "content": a}], model="gpt-4o", max_tokens=1)

In [ ]:
resp

# theory: multi-tool use?

In [ ]:
resp = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "what's the weather in philly and sf?"},
        {
            "role": "assistant",
            "content": None,
            "tool_calls": [
                {
                    "id": "foobarfoobar",
                    "type": "function",
                    "function": {"name": "get_current_weather", "arguments": '{\n  "location": "Philadelphia, PA"\n}'},
                }
            ],
        },
        {
            "name": "get_current_weather",
            "role": "tool",
            "content": create_random_string(10),
            "tool_call_id": "foobarfoobar",
        },
        {
            "role": "assistant",
            "content": None,
            "tool_calls": [
                {
                    "id": "foobarfoobar2",
                    "type": "function",
                    "function": {"name": "get_current_weather", "arguments": '{\n  "location": "San Francisco, CA"\n}'},
                }
            ],
        },
        {
            "name": "get_current_weather",
            "role": "tool",
            "content": create_random_string(10),
            "tool_call_id": "foobarfoobar2",
        },
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"},
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ],
    model="gpt-4o",
)

In [ ]:
resp

In [ ]:
resp = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "what's the weather in philly and sf?"},
        {
            "role": "assistant",
            "content": None,
            "tool_calls": [
                {
                    "id": "foobarfoobar",
                    "type": "function",
                    "function": {"name": "get_current_weather", "arguments": '{\n  "location": "Philadelphia, PA"\n}'},
                },
                {
                    "id": "foobarfoobar2",
                    "type": "function",
                    "function": {"name": "get_current_weather", "arguments": '{\n  "location": "San Francisco, CA"\n}'},
                },
            ],
        },
        {
            "name": "get_current_weather",
            "role": "tool",
            "content": create_random_string(10),
            "tool_call_id": "foobarfoobar",
        },
        {
            "name": "get_current_weather",
            "role": "tool",
            "content": create_random_string(10),
            "tool_call_id": "foobarfoobar2",
        },
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"},
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ],
    model="gpt-4o",
)
resp

In [ ]:
2240 - 2141

In [ ]:
msg_toks = 0
n_msg = 2

tool_defs = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"},
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]


def build_serial_msgs():
    msgs = []
    for n in range(n_msg):
        msgs.extend(
            [
                {
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [
                        {
                            "id": f"foobarfoobar{n}",
                            "type": "function",
                            "function": {
                                "name": "get_current_weather",
                                "arguments": '{\n  "location": "Philadelphia, PA"\n}',
                            },
                        }
                    ],
                },
                {
                    "name": "get_current_weather",
                    "role": "tool",
                    "content": create_random_string(msg_toks),
                    "tool_call_id": f"foobarfoobar{n}",
                },
            ]
        )
    return msgs


def build_parallel_msgs():
    tcs = []
    for n in range(n_msg):
        tcs.append(
            {
                "id": f"foobarfoobar{n}",
                "type": "function",
                "function": {"name": "get_current_weather", "arguments": '{\n  "location": "Philadelphia, PA"\n}'},
            }
        )

    msgs = [
        {
            "role": "assistant",
            "content": None,
            "tool_calls": tcs,
        }
    ]
    for n in range(n_msg):
        msgs.append(
            {
                "name": "get_current_weather",
                "role": "tool",
                "content": create_random_string(msg_toks),
                "tool_call_id": f"foobarfoobar{n}",
            }
        )
    return msgs


resp1 = client.chat.completions.create(
    messages=build_serial_msgs(),
    tools=tool_defs,
    model="gpt-4o",
)

resp2 = client.chat.completions.create(
    messages=build_parallel_msgs(),
    tools=tool_defs,
    model="gpt-4o",
)

print("msg toks:", msg_toks)
print("n msgs:", n_msg)
print(resp1)
print(resp2)
print("delta:", resp2.usage.prompt_tokens - resp1.usage.prompt_tokens)

In [ ]:
# 2x0       -> 12       -> 6 per
# 2x10      -> 12       -> 6 per        also    8   4 per
# 2x100     -> 18       -> 9 per        also    19  9.5 per
# 2x1000    -> 76       -> 38 per       also    99  45.5 per

# 3x0       -> 4        -> 1.33 per
# 3x10      -> 6 (???)  -> 2 per
# 3x100     -> 22       -> 7.33 per
# 3x1000    -> 124      -> 41.33 per

In [ ]:
len(encoding.encode("foobarfoobar"))